In [2]:
# Library Import

import pandas as pd
import numpy as np
import requests
import os

In [6]:
# I have already gathered existing Wind (Forecast & Actual), Demand , Interconnection data. Portal currently allows 
# user to gather data up to 2 months in the past.

# merge the dataframes

merged_df = pd.read_csv("system_demand.csv", parse_dates = ['DATE & TIME'])

work_dir = os.getcwd()

for file in os.listdir(work_dir):
    if file.endswith("tion.csv"):
        df = pd.read_csv(file, parse_dates = ['DATE & TIME'])
        merged_df = pd.merge(merged_df, df, how = 'left', on = ['DATE & TIME'])
        
# Drop & rename irrelevant columns

merged_df = merged_df.drop([' FORECAST DEMAND(MW)', ' REGION_x', ' NET TOTAL(MW)', ' REGION_y'], axis = 1)
cols = merged_df.columns.to_list()
merged_df = merged_df.rename(columns = {cols[0]: 'date_time_bst', cols[1]: 'demand_mw', cols[2]: 'ewic_mw', 
                                        cols[3]: 'moyle_mw',cols[4]: 'generation_mw', cols[5]: 'forecast_wind_mw',
                                        cols[6]: 'actual_wind_mw', cols[7]: 'region'})        